In [2]:
import pandas as pd
from scipy.interpolate import interp1d
from scipy.stats import norm
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import price_data as price
import ccxt
import time
import chart
from gql import gql, Client
from gql.transport.aiohttp import AIOHTTPTransport

# Select your transport with a defined url endpoint
transport = AIOHTTPTransport(url="https://saturn.hasura.app/v1/graphql", headers={'x-hasura-admin-secret': 'Rc07SJt4ryC6RyNXDKFRAtFmRkGBbT8Ez3SdaEYsHQoHemCldvs52Kc803oK8X62'})

# Create a GraphQL client using the defined transport
client = Client(transport=transport, fetch_schema_from_transport=True)


In [3]:
ftx = ccxt.ftx({
    'apiKey': 'mFRyLR4AAhLTc5RlWov3PKTcIbMHw3vGZwiHnsrn',
    'secret': 'oKaY1WEqTuhnNnq0iRi_Ry-CYckvE89-gPUPf21B',
    'enableRateLimit': True,
})
cex=ccxt.cex({
    'apiKey': '5ZxUkOA6EK18l15M5MVkAyoLk',
    'secret': 'qeME2eoB1MPuaLz6z51zUiWlD5A',
    'enableRateLimit': True,
})

In [4]:
results_df = pd.read_csv('data/results.csv')
results_df.iloc[results_df['success_rate'].idxmax()]

Unnamed: 0      368.0000
ema              28.0000
channel          18.0000
success_rate      0.9375
Name: 368, dtype: float64

In [5]:
float(next(filter(lambda x: x['coin']=='USD',ftx.fetch_balance()['info']['result']))['total'])

64.44664058

In [21]:
abs(np.log10(float(next(filter(lambda x:x['symbol']=='ETH/USD',ftx.fetch_markets()))['precision']['amount'])))

3.0

In [28]:
float(next(filter(lambda x: x['future']=='ETH-PERP',ftx.fetch_positions()))['size'])

0.0

In [8]:
candles=ftx.fetchOHLCV('ETH/USD','1d')
candles[-1]

[1626393600000, 1919.3, 1965.0, 1850.0, 1933.5, 242844330.1768]

In [10]:
daily=price.get_price_data('1d',symbol='ETH/USD')
hourly=price.get_price_data('1h',symbol='ETH/USD')


In [26]:
def get_rsi(candles, periods=14):
    close_delta = candles['close'].diff()

    # Make two series: one for lower closes and one for higher closes
    up = close_delta.clip(lower=0)
    down = -1 * close_delta.clip(upper=0)
    
    # Use exponential moving average
    ma_up = up.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
    ma_down = down.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
        
    rsi = ma_up / ma_down
    rsi = 100 - (100/(1 + rsi))
    return rsi
minute=price.get_price_data('1m',symbol='BTC/USD')

def get_atr(candles,periods=14):
    tr=[]
    for i in range(len(candles)):
        candle=candles.iloc[i]
        high=candle['high']
        low=candle['low']
        close=candle['close']
        true_range=max([high-low,abs(high-close),abs(close-low)])
        tr.append(true_range)
    tr=pd.Series(tr)
    atr = tr.rolling(window=periods).mean()

    return atr


1491    29.428571
1492    27.928571
1493    28.142857
1494    29.428571
1495    29.714286
1496    25.857143
1497    24.714286
1498    23.571429
1499    23.357143
1500    22.571429
dtype: float64

In [4]:
cex.fetch_balance()

AuthenticationError: requires `uid`